In [6]:
import torch
import torch.nn as nn
import numpy as np

In [10]:
vocab_size=26000
embeding_size=1024
num_heads=12
num_layers=24
batch_size=1
mid_size=1024
max_length=10
step_size=10000
epoch=1000

In [ ]:
def gelu(x):
  out = 1 + torch.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * torch.pow(x, 3)))
  return out * x / 2

In [8]:
def get_mask_matrix():
  ans=torch.triu(torch.ones(batch_size,max_length,max_length),diagonal=1)
  return ans

In [11]:
print(get_mask_matrix())

tensor([[[0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])


In [ ]:
class feedforward(nn.Module):
  def __init__(self):
    super(feedforward,self).__init__()
    self.first_layer=nn.Linear(embeding_size,mid_size)
    self.second_layer=nn.Linear(mid_size,embeding_size)
    #self.activate=nn.Tanh()
    self.layer_norm=nn.LayerNorm(embeding_size)
  def forward(self,X):
    mid_ans1=self.first_layer(X)
    mid_ans2=gelu(mid_ans1)
    mid_ans3=self.second_layer(mid_ans2)
    mid_ans4=gelu(mid_ans3)
    endans=self.layer_norm(mid_ans4)
    return endans

In [ ]:
class Attention_layer(nn.Module):
  def __init__(self):
    super(Attention_layer,self).__init__()
    self.W_Q=nn.Linear(embeding_size,embeding_size)
    self.W_K=nn.Linear(embeding_size,embeding_size)
    self.W_V=nn.Linear(embeding_size,embeding_size)
    self.W_O=nn.Linear(embeding_size,embeding_size)
  def forward(self,X):
    Q=self.W_Q(X)
    K=self.W_K(X)
    V=self.W_V(X)
    Q_heads=torch.split(Q,embeding_size//num_heads,dim=2)
    K_heads=torch.split(K,embeding_size//num_heads,dim=2)
    V_heads=torch.split(V,embeding_size//num_heads,dim=2)
    A_matrix=[]
    d_k=Q.size(-1)    Ans_matrix=[]
    mask_matrix=get_mask_matrix()
    mask_matrix1=mask_matrix.to(device)
    for i in range(len(Q_heads)):
      A_matrix.append(torch.bmm(Q_heads[i],torch.transpose(K_heads[i],1,2))+(-1e9)*mask_matrix1)
    output=[]
    for i in range(len(A_matrix)):
      output.append(torch.bmm(nn.functional.softmax(A_matrix[i],dim=2),V_heads[i])/math.sqrt(d_k))
    mid_output=torch.cat([x for x in output],2)
    #print(mid_output)
    endoutput=mid_output+self.W_O(X)
    return endoutput

In [ ]:
class GPT(nn.Module):
  def __init__(self):
    super(chineseGPT,self).__init__()
    self.attention=nn.Sequential()
    for i in range(num_layers):
      self.attention.add_module(f"attention{i}",Attention_layer())
      self.attention.add_module(f"feedworward{i}",feedforward())
  def forward(self,X):
    ans=self.attention(X)
    return ans

In [ ]:
class classifier(nn.Module):
  def __init__(self):
    super(classifier,self).__init__()
    self.first_layer=nn.Linear(embeding_size,mid_size)
    self.second_layer=nn.Linear(mid_size,embeding_size)
    self.third_layer=nn.Linear(embeding_size,vocab_size)
    #self.activate=nn.Tanh()
  def forward(self,X):
    mid_ans1=self.first_layer(X)
    mid_ans2=gelu(mid_ans1)
    mid_ans3=self.second_layer(mid_ans2)
    mid_ans4=gelu(mid_ans3)
    mid_ans5=self.third_layer(mid_ans4)
    #endans=nn.functional.softmax(mid_ans5,dim=2)
    return mid_ans5